In [ ]:
import os 
import sys
from flask import Flask, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename
import face_recognition
import cv2
import numpy as np

UPLOAD_FOLDER = '/Users/prate/upload/'
ALLOWED_EXTENSIONS = set(['txt','mp4'])
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1] in ALLOWED_EXTENSIONS

@app.route("/", methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        file = request.files['file']
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            NEWPATH=videotest(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            return render_template('video.html')
        
    return """
    <!doctype html>
    <title>Upload new File</title>
    <h1>Upload new File</h1>
    <form action="" method=post enctype=multipart/form-data>
      <p><input type=file name=file>
         <input type=submit value=Upload>
    </form>
    <p>%s</p>
    """ % "<br>".join(os.listdir(app.config['UPLOAD_FOLDER'],))



def videotest(filename):
    video_capture = cv2.VideoCapture(filename)
    length = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))

    harmesh = face_recognition.load_image_file("harmesh.jpg")
    hfencoding = face_recognition.face_encodings(harmesh)[0]

    prateek = face_recognition.load_image_file("prateek.jpeg")
    pfencoding = face_recognition.face_encodings(prateek)[0]

    # Create arrays of known face encodings and their names
    known_face_encodings = [
        hfencoding,
        pfencoding
    ]
    known_face_names = [
        "Harmesh",
        "Prateek"
    ]

    width  = int(video_capture.get(3)) # float
    height = int(video_capture.get(4))
    fourcc = cv2.VideoWriter_fourcc(*'vp80')
    PATH = '/Users/prate/static/demo.webm'
    out = cv2.VideoWriter(PATH,fourcc, fps, (width,height))
    for i in range(1,length-1):
        # Grab a single frame of video
        ret, frame = video_capture.read()

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_frame = frame[:, :, ::-1]

        # Find all the faces and face enqcodings in the frame of video
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        # Loop through each face in this frame of video
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

            name = "Unknown"

            # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 10), (right, bottom+10), (10, 10, 10), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 2, bottom), font, 0.4, (255, 255, 255), 1)

        # Display the resulting image
        #cv2.imshow('Video', frame)
        print()
        sys.stdout.write(f"writing...{int((i/length)*100)+1}%")
        sys.stdout.flush()
        out.write(frame)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release handle to the webcam
    video_capture.release()
    out.release()
    cv2.destroyAllWindows()
    return PATH

    
if __name__ == "__main__":
    app.run(host='127.0.0.1', port=5001, debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Jun/2020 20:02:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2020 20:02:40] "GET / HTTP/1.1" 200 -



writing...1%
writing...2%
writing...3%
writing...4%
writing...5%
writing...6%
writing...7%
writing...8%
writing...9%
writing...10%
writing...10%
writing...11%
writing...12%
writing...13%
writing...14%
writing...15%
writing...16%
writing...17%
writing...18%
writing...19%
writing...19%
writing...20%
writing...21%
writing...22%
writing...23%
writing...24%
writing...25%
writing...26%
writing...27%
writing...28%
writing...28%
writing...29%
writing...30%
writing...31%
writing...32%
writing...33%
writing...34%
writing...35%
writing...36%
writing...37%
writing...37%
writing...38%
writing...39%
writing...40%
writing...41%
writing...42%
writing...43%
writing...44%
writing...45%
writing...46%
writing...46%
writing...47%
writing...48%
writing...49%
writing...50%
writing...51%
writing...52%
writing...53%
writing...54%
writing...55%
writing...55%
writing...56%
writing...57%
writing...58%
writing...59%
writing...60%
writing...61%
writing...62%
writing...63%
writing...64%
writing...64%
writing...65%


127.0.0.1 - - [30/Jun/2020 20:05:56] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Jun/2020 20:05:56] "GET /static/demo.webm HTTP/1.1" 206 -
127.0.0.1 - - [30/Jun/2020 20:05:56] "GET /static/demo.webm HTTP/1.1" 206 -
127.0.0.1 - - [30/Jun/2020 20:05:57] "GET /static/demo.webm HTTP/1.1" 206 -
127.0.0.1 - - [30/Jun/2020 20:05:57] "GET /static/demo.webm HTTP/1.1" 206 -
127.0.0.1 - - [30/Jun/2020 20:05:57] "GET /static/demo.webm HTTP/1.1" 206 -
127.0.0.1 - - [30/Jun/2020 20:05:57] "GET /static/demo.webm HTTP/1.1" 206 -
127.0.0.1 - - [30/Jun/2020 20:05:57] "GET /static/demo.webm HTTP/1.1" 206 -
127.0.0.1 - - [30/Jun/2020 20:05:57] "GET /static/demo.webm HTTP/1.1" 206 -
127.0.0.1 - - [30/Jun/2020 20:05:57] "GET /static/demo.webm HTTP/1.1" 206 -
127.0.0.1 - - [30/Jun/2020 20:06:02] "GET /static/demo.webm HTTP/1.1" 206 -
